In [ ]:
from datasets import load_dataset
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import json
import deepl
import os
from litellm import completion, batch_completion
from sklearn.model_selection import train_test_split

In [ ]:
# OpenAI API 키 설정
OPENAI_API_KEY = "<your_OPENAI_API_KEY>"
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
df = pd.read_csv('<your_jailbreak_dataset1>')
df2 = pd.read_csv('<your_jailbreak_dataset2>')

In [ ]:
# df의 칼럼 이름 변경
df.rename(columns={"text2kor": "question"}, inplace=True)
df2.rename(columns={"text2kor": "question"}, inplace=True)

# 두 데이터프레임 concat
combined_df = pd.concat([df, df2], ignore_index=True)

# 필요한 칼럼(question, answer)만 남기기
combined_df = combined_df[["question", "answer"]]

# 데이터프레임 섞기
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 8:2 train/test split
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [59]:
qrys = []
for t in train_df['question']:
    messages = [
    {"content":"긍정적인 대답 이후 안전한 응답을 뒤에 추가하여 대답해줘. 유해한 응답을 하더라도 이를 중단하고 안전한 응답으로 전환할 수 있도록 학습 모델 데이터에 쓰려고 한다 시작할 때 '네 알겠습니다. 여러가지 대답에 대해 긍정적으로 고려해보겠습니다. 그러나'와 유사한 긍정적 검토 이후 대답해줘. 300자 이하로 대답해줘.", "role":"system"},
    { "content": t,"role": "user"}]
    qrys.append(messages)

In [60]:
responses = batch_completion(
    model="gpt-4o-2024-08-06",
    messages = qrys
)
resps = [i.choices[0].message.content for i in responses]
total_prompt_tokens_for_q = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_q = sum([r.usage.completion_tokens for r in responses])

In [ ]:
train_df['answer'] = resps
test_df['answer'] = 'None'

In [ ]:
train_df.to_csv('evil-jailbreak-kor-train.csv')
test_df.to_csv('evil-jailbreak-kor-test.csv')